In [ ]:
# Install necessary dependencies
!sudo apt-get update
!sudo apt-get install -y ffmpeg
!wget https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp -O /usr/local/bin/yt-dlp
!chmod a+rx /usr/local/bin/yt-dlp

!git clone https://github.com/rxys/Impractical-RIFE.git
%cd Impractical-RIFE
!pip install -r requirements.txt

# Download RIFEv4.26 weights
!pip install gdown
!gdown --id 1gViYvvQrtETBgU1w8axZSsr7YUuw31uy -O RIFEv4.26_0921.zip
!unzip -o RIFEv4.26_0921.zip -d .
%cd ..

In [ ]:
# Import libraries
import os
import shutil
import cv2
from google.colab import files


# Clean up any leftover files from previous runs (except final output videos)
tmp_dir = os.path.abspath("interp_videos")
if os.path.exists(tmp_dir):
    shutil.rmtree(tmp_dir)
os.makedirs(tmp_dir, exist_ok=True)
final_destination = os.path.abspath("final_videos")
os.makedirs(final_destination, exist_ok=True)

# Cleanup previous video downloads except final output videos
for file in os.listdir():
  if file.endswith(".mp4"):
    os.remove(file)

# Input YouTube URL directly in the cell
youtube_url = input("Enter YouTube URL: ")

# Run youtube-dl in shell using ! notation
!yt-dlp -f bestvideo+bestaudio --merge-output-format mp4 {youtube_url}

# Get downloaded file name
video_files = [f for f in os.listdir() if f.endswith(".mp4")]
assert len(video_files) == 1, "Unexpected number of video files downloaded."
input_video = video_files[0]

# Set final output video name
final_output_name = os.path.join(final_destination, os.path.splitext(input_video)[0] + "_60.mp4")

print(f"Downloaded video: {input_video}")

# Rename the downloaded video to avoid breaking RIFE command
renamed_video = os.path.abspath("temp_video.mp4")
os.rename(input_video, renamed_video)

# Trim video to first 10 seconds using ffmpeg
trimmed_video = "trimmed_video.mp4"
!ffmpeg -i {renamed_video} -t 10 -c copy {trimmed_video}
os.remove(renamed_video)
renamed_video = os.path.abspath(trimmed_video)

# Extract frame rate
video = cv2.VideoCapture(renamed_video)
fps = video.get(cv2.CAP_PROP_FPS)
print(f"Original FPS: {fps}")
video.release()

# Determine interpolation multiplier and target fps
if 23.97 <= fps <= 24.0:
    multi = 5
    drop = 2
    out_fps = fps * multi / 2
elif 29.97 <= fps <= 30.0:
    multi = 2
    drop = 1
    out_fps = fps * multi
else:
    raise ValueError(f"Unsupported FPS: {fps}. This script supports ~24fps and ~30fps videos.")


# Get the absolute path of the RIFE directory
rife_dir = os.path.abspath("Impractical-RIFE")
interp_video = f"{tmp_dir}/output.mp4"
combined_video = f"{tmp_dir}/combined.mp4"
# For other fps, interpolate as usual and produce a video
os.chdir(rife_dir)
!python inference_video.py --multi={multi} --drop={drop} --fps={out_fps} --video={renamed_video} --output={interp_video}
os.chdir("..")

# Encode final video with source audio copied
!ffmpeg -i {interp_video} -i {renamed_video} -map 0:v -map 1:a -c:v copy -c:a copy {combined_video}

shutil.move(combined_video, final_output_name)
os.remove(renamed_video)

# Provide the final video for download
print(f"Final video saved as {final_output_name}")
files.download(final_output_name)
